In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        pass

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# first trial
# classification of dataset 'cat-and-dog'
# reference: https://www.kaggle.com/code/upf1sh/made-by-pytorch-best-valid-acc92-5
# pytorch, cnn, classification

In [3]:
import torch
import torch.nn as nn
from PIL import Image  # originate images
from torchvision import datasets
from torch.utils.data import DataLoader
# parameter: batchsize, shuffle(epoch乱序), num_workers(加载子进程)
from torchvision import transforms  # image transformation

In [4]:
train_dir = '/kaggle/input/cat-and-dog/training_set/training_set'
test_dir = '/kaggle/input/cat-and-dog/test_set/test_set'

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])
data_transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    # update image size with crop at the centre of 256 size of image
    transforms.ToTensor(),
    normalize  # normalize the image of train and test
])

train_dataset = datasets.ImageFolder(train_dir, transform=data_transform)
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.ImageFolder(test_dir, transform=data_transform)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=True)

In [5]:
# CNN(could be updated)
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        #  padding=(k-1)/2, size without change
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, 1, 1),  # (16,224,224)
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(2,2),  # (16,112,112)
            
            nn.Conv2d(16, 32, 3, 1, 1),  # (32,112,112)
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2,2),  # (32,56,56)
            
            nn.Conv2d(32, 64, 3, 1, 1),  # (64,56,56)
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2,2),  # (64,28,28)
            
            nn.Conv2d(64, 128, 3, 1, 1),  # (128,28,28)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2,2),  # (128,14,14)
            
            nn.Conv2d(128, 256, 3, 1, 1),  # (256,14,14)
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2,2),  # (256,7,7)
            
        )
        
        self.fc = nn.Sequential(
            nn.Linear(256*7*7, 1024),
            nn.ReLU(),
            nn.Linear(1024, 128),
            nn.ReLU(),
            nn.Linear(128, 2),
        )
    
    def forward(self, x):
        out = self.conv(x)
        out = out.view(out.size()[0], -1)  # reshape
        return self.fc(out)

In [6]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Net().to(device)
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# .parameters() store weights, return an iterator

def train_loop(dataloader, model, loss, optimizer):
    size = len(dataloader.dataset)
    model.train()  # in mode 'train', using BN and Dropout
    for batch, (X,y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        pred = model(X)  # prediction
        temploss = loss(pred, y)
        optimizer.zero_grad()  # clear old grad
        temploss.backward()  # compute present grad
        optimizer.step()  # update parameters with grad
        if batch % 10 == 0:
            temploss, current = temploss.item(), batch * len(X)
            print(f'loss:{temploss:>7f} [{current:>5d}/{size:>5d}]')

In [7]:
best_acc, epochs = 0, 10
for epoch in range(epochs):
    print(f'Epoch{epoch+1}\n------------------')
    train_loop(train_loader, model, loss, optimizer)
    
    print('test{}'.format(epoch))
    size = len(test_loader.dataset)
    num_batches = len(test_loader)
    model.eval()  # in mode 'test', without change parameters
    test_loss, correct = 0, 0
    with torch.no_grad():  # reduce RAM, without change pars
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss(pred, y).item()  # read value of tensor
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f'Test Error: \n Accuracy: {(100 * correct):>0.1f}%\n')
    # save model
    patience = 7
    if correct > best_acc:
        print(f'Best model found at epoch {epoch+1}, saving model')
        torch.save(model.state_dict(), 'clsf_cd.pt')
        best_acc = correct
        stale = 0
    else:
        stale += 1
        if stale > patience:
            print(f'No improvment {patience} consecutive epochs, early stopping')
            break

Epoch1
------------------
loss:0.709052 [    0/ 8005]
loss:0.667794 [  640/ 8005]
loss:0.736919 [ 1280/ 8005]
loss:0.602694 [ 1920/ 8005]
loss:0.532791 [ 2560/ 8005]
loss:0.546004 [ 3200/ 8005]
loss:0.590218 [ 3840/ 8005]
loss:0.650969 [ 4480/ 8005]
loss:0.520267 [ 5120/ 8005]
loss:0.614077 [ 5760/ 8005]
loss:0.581073 [ 6400/ 8005]
loss:0.554069 [ 7040/ 8005]
loss:0.691557 [ 7680/ 8005]
test0
Test Error: 
 Accuracy: 67.6%

Best model found at epoch 1, saving model
Epoch2
------------------
loss:0.526932 [    0/ 8005]
loss:0.552473 [  640/ 8005]
loss:0.538199 [ 1280/ 8005]
loss:0.406033 [ 1920/ 8005]
loss:0.516084 [ 2560/ 8005]
loss:0.593429 [ 3200/ 8005]
loss:0.503329 [ 3840/ 8005]
loss:0.513849 [ 4480/ 8005]
loss:0.501019 [ 5120/ 8005]
loss:0.497897 [ 5760/ 8005]
loss:0.527191 [ 6400/ 8005]
loss:0.598866 [ 7040/ 8005]
loss:0.480135 [ 7680/ 8005]
test1
Test Error: 
 Accuracy: 70.6%

Best model found at epoch 2, saving model
Epoch3
------------------
loss:0.422913 [    0/ 8005]
loss:0.5